In [1]:
import pandas as pd
from dotenv import load_dotenv
import os
import pandas as pd
from IPython.display import HTML
import sys

project_root = os.path.dirname(os.getcwd())
sys.path.append(project_root)

from src.classifiers import is_uae_real_estate_query, llm_classifier
from src.tools import visualize_tool, safe_dataframe_tool
from src.geo_tools import generate_google_maps_html

load_dotenv()

True

In [2]:
def main_agent(query: str):
    if not is_uae_real_estate_query(query):
        return {"type": "output", "data": "This is an irrelevant question to UAE property."}

    data = safe_dataframe_tool(query)
    if isinstance(data, dict) and data.get("error"):
        return data

    action = llm_classifier(query)

    if action == "output":
        return {"type": "data", "data": data}

    elif action == "plot_stats":
        result = visualize_tool.invoke({"data": data, "query": query})
        return {"type": "plot", **result}

    elif action == "geospatial_plot":
        result = generate_google_maps_html(data)
        return {"type": "html","content": result}

    else:
        return {"error": f"Unknown action '{action}' from classifier."}


In [3]:
result = main_agent("Locate on map properties from Dubai under 2 million AED")

/tmp/ipykernel_122880/1533224408.py:5: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use :meth:`~invoke` instead.
  data = safe_dataframe_tool(query)


                                                 title  \
2            Fully Furnished | Smart Home | High Floor   
7    DSO Landscape View | x3 Parking | Post Handove...   
11                  Fully Upgraded | Luxury | High-ROI   
58   VACANT | BEST Deal on the market | Biggest layout   
61          Great ROI | 1BR Furnished | Prime Location   
..                                                 ...   
530           Canal View | Mid Floor | Spacious Layout   
537           LUXURIOUS 2 BR | READY UNIT | BEST VALUE   
547                 VACANT / ALL BRANDNEW / GUARANTEED   
548     Vacant | Great ROI | Brand New | Marina Access   
553   Vacant |Sea View |Kitchen Equipped |Mid Floor  |   

                                 displayAddress  bathrooms  bedrooms  \
2                     East 40, Al Furjan, Dubai          1         1   
7    Tria By Deyaar, Dubai Silicon Oasis, Dubai          4         2   
11   Oxford 212, Jumeirah Village Circle, Dubai          1         1   
58           Oc

In [6]:
result2 = main_agent("List properties from Dubai under 2 million AED")

output


In [8]:
pd.DataFrame(data=result2["data"])

,title,displayAddress,bathrooms,bedrooms,addedOn,type,price,verified,priceDuration,sizeMin,furnishing,description,latitude,longitude
0,Fully Furnished | Smart Home | High Floor,"East 40, Al Furjan, Dubai",1.0,1.0,2024-08-15 10:05:41+00:00,Residential for Sale,950000,True,sell,NaN,YES,<p>Coldwell banker is pleased to present to yo...,25.021090,55.140395
1,DSO Landscape View | x3 Parking | Post Handove...,"Tria By Deyaar, Dubai Silicon Oasis, Dubai",4.0,2.0,2024-08-06 19:15:20+00:00,Residential for Sale,1790000,True,sell,NaN,PARTLY,First Bridge Real Estate are pleased to presen...,25.124155,55.373460
2,Fully Upgraded | Luxury | High-ROI,"Oxford 212, Jumeirah Village Circle, Dubai",1.0,1.0,2024-08-08 08:56:26+00:00,Residential for Sale,1350000,True,sell,NaN,NO,Azco Real Estate is pleased to offer this styl...,25.056619,55.201522
3,FREEHOLD VILLA 3 BED -5%DP-CRYSTAL LAGOONS,"Hayyan, Sharjah",4.0,3.0,2024-07-27 07:44:07+00:00,Residential for Sale,1990000,False,sell,NaN,PARTLY,Hayyan is a new cultural nexus on the edge of ...,25.048561,56.350046
4,VACANT | BEST Deal on the market | Biggest layout,"Ocean Heights, Dubai Marina, Dubai",2.0,1.0,2024-06-07 05:24:47+00:00,Residential for Sale,1380000,True,sell,NaN,YES,Betterhomes is offering this stunning 1-bedroo...,25.090666,55.148777
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154,Vacant | Great ROI | Brand New | Marina Access,"Marina Star, Dubai Marina, Dubai",1.0,NaN,2024-07-02 09:29:08+00:00,Residential for Sale,1400000,True,sell,NaN,NO,RB Homes is proud to bring to the residential ...,25.068635,55.131475
155,Amazing Layout | Well Maintained | Ready to Move,"Tower 1, Al Reef Downtown, Al Reef, Abu Dhabi",2.0,2.0,2024-06-01 12:09:49+00:00,Residential for Sale,730000,True,sell,NaN,NO,Al Reef is a mixed-use development spanning ov...,24.456114,54.672554
156,Modern Living | Amazing Offer | Prime Location,"Park View, Saadiyat Island, Abu Dhabi",1.0,NaN,2024-05-20 09:05:32+00:00,Residential for Sale,730000,True,sell,NaN,NO,Park View offers an exhilarating range of amen...,24.523703,54.432280
157,Vacant |Sea View |Kitchen Equipped |Mid Floor |,"Botanica Tower, Dubai Marina, Dubai",2.0,1.0,2024-07-30 08:38:11+00:00,Residential for Sale,1600000,True,sell,NaN,NO,Call Hassan for more information and viewing +...,25.085303,55.142384


In [26]:
df_result["addedOn"].max()

Timestamp('2024-08-18 06:46:37+0000', tz='UTC')

In [27]:
df_result["addedOn"].min()

Timestamp('2023-08-25 22:10:36+0000', tz='UTC')

In [9]:
code2 = generate_google_maps_html(result2["data"])

In [10]:
with open("property_map_test2.html", "w", encoding="utf-8") as f:
    f.write(code2)

In [5]:
with open("property_map_test1.html", "w", encoding="utf-8") as f:
    f.write(result["content"])

In [13]:
df = pd.read_parquet(r"../data/uae_real_estate_2024_geo_ready.parquet")

In [14]:
df

,title,displayAddress,bathrooms,bedrooms,addedOn,type,price,verified,priceDuration,sizeMin,furnishing,description,latitude,longitude
0,Great ROI I High Floor I Creek View,"Binghatti Canal, Business Bay, Dubai",3,2,2024-08-14 12:02:53+00:00,Residential for Sale,2500000,True,sell,NaN,NO,MNA Properties is delighted to offer this apar...,25.186109,55.286156
1,Full Sea View | Beach Life | Brand New Residence,"La Vie, Jumeirah Beach Residence, Dubai",3,2,2024-08-13 05:45:16+00:00,Residential for Sale,7200000,True,sell,NaN,YES,"Apartment for sale in La Vie, Jumeirah Beach R...",25.079735,55.134366
2,Fully Furnished | Smart Home | High Floor,"East 40, Al Furjan, Dubai",1,1,2024-08-15 10:05:41+00:00,Residential for Sale,950000,True,sell,NaN,YES,<p>Coldwell banker is pleased to present to yo...,25.021090,55.140395
3,VOT / Single Row / Private Pool / Landscaped /...,"Meadows 3, Meadows, Dubai",5,5,2024-08-15 12:58:08+00:00,Residential for Sale,9999990,True,sell,NaN,YES,&#34; 1 Day Prior Notice required &#34; &#34;...,25.070594,55.157207
4,Sea View | Move In Ready | Great Location,"Princess Tower, Dubai Marina, Dubai",3,2,2024-08-07 10:32:26+00:00,Residential for Sale,2100000,True,sell,NaN,YES,"GTA Properties is proud to present this 2-bed,...",25.088709,55.146727
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
558,Upgraded | Corner Unit | Near Lake | Single Row,"Zulal 1, Zulal, The Lakes, Dubai",3,3,2024-06-13 12:54:16+00:00,Residential for Sale,5750000,True,sell,NaN,NO,The Urban Nest is pleased to present a 3-bedro...,25.079259,55.174432
559,Marina View | High Floor | Unfurnished,"Cayan Tower, Dubai Marina, Dubai",3,2,2023-09-01 10:43:10+00:00,Residential for Sale,2650000,True,sell,NaN,NO,Rocky Real Estate is proud to present this mar...,25.086846,55.145171
560,Invest Now| Amazing Plot| High ROI| Prime Area,"West Yas, Yas Island, Abu Dhabi",<NA>,<NA>,2024-06-21 11:43:02+00:00,Residential for Sale,5200000,False,sell,NaN,NO,Seize this excellent opportunity to own an ama...,24.494282,54.583130
561,Emaar | Waterfront Apartment | 10% DP | 5-Yr PP,"Al Marjan Island, Ras Al Khaimah",2,1,2024-08-12 13:08:33+00:00,Residential for Sale,3007888,False,sell,NaN,NO,"Discover Address Residence by Emaar, nestled o...",25.677675,55.739598


In [16]:
df["displayAddress"].iloc[0].split(",")[-1].strip()

'Dubai'

In [17]:
df["City"] = df["displayAddress"].apply(lambda x: x.split(",")[-1].strip())

In [21]:
df.to_parquet(r"../data/uae_real_estate_2024_geo_ready2.parquet", index=False)